In [20]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils import parameters_to_vector
from torch.optim import Adam
from preds.models import MLPS
from preds.likelihoods import GaussianLh
from preds.datasets import UCIClassificationDatasets
from preds.laplace import Laplace

In [21]:
width = 25
depth = 3
prior_prec = 0.1
lr = 5e-3
n_epochs = 1000
lh = GaussianLh(sigma_noise=0.3)  # fixed to true noise
uci_dataset = 'glass'
root_dir = '/Users/tamire1/Documents/GitHub/BNN-predictions/data/'
device = 'cpu'

Load training data

In [22]:
ds_train = UCIClassificationDatasets(train=True, data_set=uci_dataset, split_train_size=0.95, double=False, root=root_dir)
X_train, y_train = ds_train.data.to(device), ds_train.targets.to(device).unsqueeze(1)
train_loader = [(X_train, y_train)]  # mock

Load validation data

In [23]:
ds_val = UCIClassificationDatasets(train=False,valid=True, data_set=uci_dataset, split_train_size=0.95, double=False, root=root_dir)
X_val, y_val = ds_val.data.to(device), ds_val.targets.to(device).unsqueeze(1)
val_loader = [(X_val, y_val)] 

In [24]:
model = MLPS(X_train.shape[1], [width]*depth, 1, activation='tanh', flatten=False).to(device)
optim = Adam(model.parameters(), lr=lr)
losses = list()
for i in range(n_epochs):
    f = model(X_train)
    w = parameters_to_vector(model.parameters())
    reg = 0.5 * prior_prec * w @ w
    loss = - lh.log_likelihood(y_train, f) + reg
    loss.backward()
    optim.step()
    losses.append(loss.item())
    model.zero_grad()
# optionally: plt.plot(losses) to monitor convergence

In [25]:
lap = Laplace(model, float(prior_prec), lh)


def get_pred_for(x, model_type='glm', cov_type='full'):
    #### INFERENCE (Posterior approximation) ####
    lap.infer(train_loader, cov_type=cov_type, dampen_kron=model_type=='bnn')
    if model_type == 'glm':
        #### GLM PREDICTIVE ####
        mu, var = lap.predictive_samples_glm(x, n_samples=1000)
    elif model_type == 'bnn':
        #### BNN PREDICTIVE ####
        samples = lap.predictive_samples_bnn(x, n_samples=1000)
        mu, var = samples.mean(dim=0), samples.var(dim=0)
    else:
        raise ValueError('unsupported model_type.')
    mu = mu.detach().cpu().squeeze().numpy()
    var = var.detach().cpu().squeeze().numpy()
    return mu, var

In [30]:
print(X_train.dtype)
print(X_val.dtype)
print()

torch.float32

In [26]:
# GLM

mu, var = get_pred_for(X_val, 'glm', 'full')
mu, var = get_pred_for(X_val, 'glm', 'kron')
mu, var = get_pred_for(X_val, 'glm', 'diag')

RuntimeError: Found dtype Long but expected Float
Exception raised from compute_types at /Users/distiller/project/conda/conda-bld/pytorch_1595629430416/work/aten/src/ATen/native/TensorIterator.cpp:183 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >) + 169 (0x10f2f9199 in libc10.dylib)
frame #1: at::TensorIterator::compute_types(at::TensorIteratorConfig const&) + 3842 (0x134695312 in libtorch_cpu.dylib)
frame #2: at::TensorIterator::build(at::TensorIteratorConfig&) + 618 (0x13469e51a in libtorch_cpu.dylib)
frame #3: at::TensorIterator::TensorIterator(at::TensorIteratorConfig&) + 223 (0x13469e1ff in libtorch_cpu.dylib)
frame #4: at::native::mse_loss_backward_out(at::Tensor&, at::Tensor const&, at::Tensor const&, at::Tensor const&, long long) + 410 (0x1344e9f7a in libtorch_cpu.dylib)
frame #5: at::CPUType::mse_loss_backward_out_grad_input(at::Tensor&, at::Tensor const&, at::Ten